In [1]:
import pandas as pd
import numpy as np
import tweepy
import requests
import re
import json
import matplotlib.pyplot as plt
import datetime
import os
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

# Gathering Data:

In [2]:
# read twitter_archive as a Pandas DataFrame
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

#Downloading and save the image prediction data using requests
url="https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
response = requests.get(url)
url_list=url.split("/")#extracting nane of file programmatically
file_name=url_list[-1]
with open(file_name, 'wb') as file:
    file.write(response.content)
image_predictions = pd.read_csv(file_name, sep='\t')


In [3]:
twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [4]:
twitter_archive.tail()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None
2355,666020888022790149,NaN,NaN,2015-11-15 22:32:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a Japanese Irish Setter. Lost eye...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666020888...,8,10,None,None,None,None,None


In [5]:
twitter_archive.sample(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
257,843856843873095681,NaN,NaN,2017-03-20 16:08:44 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Sadie and Daisy. They do all thei...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/843856843...,12,10,Sadie,None,None,None,None
478,815966073409433600,NaN,NaN,2017-01-02 17:00:46 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a pupper with squeaky hiccups. Please e...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/815966073...,13,10,None,None,None,pupper,None
203,853299958564483072,NaN,NaN,2017-04-15 17:32:18 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Rumpole. He'll be your Uber driver thi...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/853299958...,13,10,Rumpole,None,None,None,None
761,778286810187399168,NaN,NaN,2016-09-20 17:36:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Stanley. He has too much skin. Isn't h...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/778286810...,11,10,Stanley,None,None,None,None
1733,679777920601223168,NaN,NaN,2015-12-23 21:37:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Rinna. She's melting. 10/10 get inside...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/679777920...,10,10,Rinna,None,None,pupper,None
258,843604394117681152,NaN,NaN,2017-03-19 23:25:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Hank. He's been outside for 3 minutes ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/843604394...,11,10,Hank,None,None,None,None
1099,735991953473572864,NaN,NaN,2016-05-27 00:32:10 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Maxaroni. He's curly af. Also rather f...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/735991953...,11,10,Maxaroni,None,None,None,None
2175,669214165781868544,NaN,NaN,2015-11-24 18:01:05 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Jaspers. He is a northeastern Gillette...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/669214165...,10,10,Jaspers,None,None,None,None
1260,710269109699739648,NaN,NaN,2016-03-17 00:58:46 +0000,"<a href=""http://twitter.com/download/iphone"" r...",The squad is back for St. Patrick's Day! ☘ 💚\n...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/710269109...,13,10,None,None,None,None,None
133,866720684873056260,NaN,NaN,2017-05-22 18:21:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...",He was providing for his family 13/10 how dare...,NaN,NaN,NaN,https://twitter.com/nbcnews/status/86645871888...,13,10,None,None,None,None,None


In [6]:
image_predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [7]:
image_predictions.sample(100)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
247,670668383499735048,https://pbs.twimg.com/media/CU6xVkbWsAAeHeU.jpg,1,banana,0.107317,False,orange,0.099662,False,bagel,0.089033,False
1913,854010172552949760,https://pbs.twimg.com/media/C9oNt91WAAAFSLS.jpg,1,English_springer,0.354733,True,collie,0.177538,True,Border_collie,0.131706,True
1545,792050063153438720,https://pbs.twimg.com/media/Cv3tU38WcAASFas.jpg,2,komondor,0.942856,True,swab,0.052715,False,Tibetan_terrier,0.002743,True
657,682303737705140231,https://pbs.twimg.com/media/CXgHoLnWAAA8i52.jpg,1,seat_belt,0.997659,False,Lakeland_terrier,0.001731,True,Airedale,0.000204,True
984,707420581654872064,https://pbs.twimg.com/media/CdFDQVgWIAArslx.jpg,1,ram,0.518215,False,kuvasz,0.149391,True,Great_Pyrenees,0.106003,True
...,...,...,...,...,...,...,...,...,...,...,...,...
741,687317306314240000,https://pbs.twimg.com/media/CYnXcLEUkAAIQOM.jpg,1,Shih-Tzu,0.747208,True,Maltese_dog,0.091025,True,Lhasa,0.035788,True
308,671536543010570240,https://pbs.twimg.com/media/CVHG6_AWwAEJf_u.jpg,1,pug,0.537652,True,bull_mastiff,0.220617,True,French_bulldog,0.068297,True
1734,821522889702862852,https://pbs.twimg.com/media/C2aitIUXAAAG-Wi.jpg,1,Doberman,0.763539,True,black-and-tan_coonhound,0.136602,True,miniature_pinscher,0.087654,True
1864,842892208864923648,https://pbs.twimg.com/ext_tw_video_thumb/80710...,1,Chihuahua,0.505370,True,Pomeranian,0.120358,True,toy_terrier,0.077008,True


In [8]:
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [9]:
#Query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's
#entire set of JSON data in a file.

In [10]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)


In [11]:
# Read the saved tweet_json.txt file into a dataframe
tweet_data = pd.read_csv('tweet-json.txt',error_bad_lines=False)
#tweet_data=tweet-json.to_csv('tweet-json.txt', encoding = 'utf-8', index=False)
tweet_data.head()

b'Skipping line 5: expected 126 fields, saw 146\nSkipping line 8: expected 126 fields, saw 144\nSkipping line 11: expected 126 fields, saw 146\nSkipping line 14: expected 126 fields, saw 144\nSkipping line 16: expected 126 fields, saw 133\nSkipping line 18: expected 126 fields, saw 144\nSkipping line 19: expected 126 fields, saw 188\nSkipping line 20: expected 126 fields, saw 144\nSkipping line 22: expected 126 fields, saw 134\nSkipping line 23: expected 126 fields, saw 144\nSkipping line 24: expected 126 fields, saw 137\nSkipping line 26: expected 126 fields, saw 144\nSkipping line 27: expected 126 fields, saw 149\nSkipping line 29: expected 126 fields, saw 144\nSkipping line 32: expected 126 fields, saw 307\nSkipping line 35: expected 126 fields, saw 220\nSkipping line 36: expected 126 fields, saw 305\nSkipping line 37: expected 126 fields, saw 188\nSkipping line 39: expected 126 fields, saw 188\nSkipping line 42: expected 126 fields, saw 222\nSkipping line 45: expected 126 fields, s

,,,,"{""created_at"": ""Tue Aug 01 16:23:56 +0000 2017""","""id"": 892420643555336193","""id_str"": ""892420643555336193""","""full_text"": ""This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU""","""truncated"": false","""display_text_range"": [0",85],"""entities"": {""hashtags"": []","""symbols"": []","""user_mentions"": []",...,"""place"": null","""contributors"": null","""is_quote_status"": false","""retweet_count"": 8853","""favorite_count"": 39467","""favorited"": false","""retweeted"": false","""possibly_sensitive"": false","""possibly_sensitive_appealable"": false","""lang"": ""en""}"
"{""created_at"": ""Tue Aug 01 00:17:27 +0000 2017""","""id"": 892177421306343426","""id_str"": ""892177421306343426""","""full_text"": ""This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not",she's available for pats,snugs,boops,"the whole bit. 13/10 https://t.co/0Xxu71qeIV""","""truncated"": false","""display_text_range"": [0",138],"""entities"": {""hashtags"": []","""symbols"": []","""user_mentions"": []",...,"""place"": null","""contributors"": null","""is_quote_status"": false","""retweet_count"": 6514","""favorite_count"": 33819","""favorited"": false","""retweeted"": false","""possibly_sensitive"": false","""possibly_sensitive_appealable"": false","""lang"": ""en""}"
"{""created_at"": ""Mon Jul 31 00:18:03 +0000 2017""","""id"": 891815181378084864","""id_str"": ""891815181378084864""","""full_text"": ""This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB""","""truncated"": false","""display_text_range"": [0",121],"""entities"": {""hashtags"": []","""symbols"": []","""user_mentions"": []","""urls"": []","""media"": [{""id"": 891815175371796480","""id_str"": ""891815175371796480""","""indices"": [122",...,"""favorite_count"": 25461","""favorited"": false","""retweeted"": false","""possibly_sensitive"": false","""possibly_sensitive_appealable"": false","""lang"": ""en""}",NaN,NaN,NaN,NaN
"{""created_at"": ""Sun Jul 30 15:58:51 +0000 2017""","""id"": 891689557279858688","""id_str"": ""891689557279858688""","""full_text"": ""This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ""","""truncated"": false","""display_text_range"": [0",79],"""entities"": {""hashtags"": []","""symbols"": []","""user_mentions"": []","""urls"": []","""media"": [{""id"": 891689552724799489","""id_str"": ""891689552724799489""","""indices"": [80",...,"""favorite_count"": 42908","""favorited"": false","""retweeted"": false","""possibly_sensitive"": false","""possibly_sensitive_appealable"": false","""lang"": ""en""}",NaN,NaN,NaN,NaN
"{""created_at"": ""Sat Jul 29 00:08:17 +0000 2017""","""id"": 891087950875897856","""id_str"": ""891087950875897856""","""full_text"": ""Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh""","""truncated"": false","""display_text_range"": [0",138],"""entities"": {""hashtags"": [{""text"": ""BarkWeek""","""indices"": [129",138]}],"""symbols"": []","""user_mentions"": []","""urls"": []","""media"": [{""id"": 891087942176911360",...,"""is_quote_status"": false","""retweet_count"": 3261","""favorite_count"": 20562","""favorited"": false","""retweeted"": false","""possibly_sensitive"": false","""possibly_sensitive_appealable"": false","""lang"": ""en""}",NaN,NaN
"{""created_at"": ""Fri Jul 28 16:27:12 +0000 2017""","""id"": 890971913173991426","""id_str"": ""890971913173991426""","""full_text"": ""Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below\n\nhttps://t.co/Zr4hWfAs1H https://t.co/tVJBRMnhxl""","""truncated"": false","""display_text_range"": [0",140],"""entities"": {""hashtags"": []","""symbols"": []","""user_mentions"": []","""urls"": [{"

In [12]:
tweet_id_df = []

with open('tweet-json.txt', 'r') as file:
    for line in file:
        tweet = json.loads(line)
        tweet_id = tweet['id']
        retweet_count = tweet['retweet_count']
        fav_count = tweet['favorite_count']
        user_followers = tweet['user']['followers_count'] 
        date_time = tweet['created_at'] 
        user_favourites = tweet['user']['favourites_count'] 
        #possibly_sensitive=tweet['possibly_sensitive']
        #print(type(possibly_sensitive))
        tweet_id_df.append({'tweet_id':tweet_id,
                            'favorites': int(fav_count),
                            'retweets': int(retweet_count),
                            'user_followers': int(user_followers),
                            'user_favourites': int(user_favourites),
                            'date_time': pd.to_datetime(date_time),
                            'favorite_count': fav_count})
        
        
api_df_now = pd.DataFrame(tweet_id_df)
api_df_now.head()



,tweet_id,favorites,retweets,user_followers,user_favourites,date_time,favorite_count
0,892420643555336193,39467,8853,3200889,114031,2017-08-01 16:23:56+00:00,39467
1,892177421306343426,33819,6514,3200889,114031,2017-08-01 00:17:27+00:00,33819
2,891815181378084864,25461,4328,3200889,114031,2017-07-31 00:18:03+00:00,25461
3,891689557279858688,42908,8964,3200889,114031,2017-07-30 15:58:51+00:00,42908
4,891327558926688256,41048,9774,3200889,114031,2017-07-29 16:00:24+00:00,41048


# Assessing Data:

In [13]:
print("Twitter _archive sample","\n",twitter_archive.sample(5),"\n")
twitter_archive.info()

Twitter _archive sample 
                 tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
1873  675145476954566656                    NaN                  NaN   
2028  671866342182637568                    NaN                  NaN   
201   853760880890318849                    NaN                  NaN   
62    880095782870896641                    NaN                  NaN   
1611  685325112850124800                    NaN                  NaN   

                      timestamp  \
1873  2015-12-11 02:49:59 +0000   
2028  2015-12-02 01:39:53 +0000   
201   2017-04-17 00:03:50 +0000   
62    2017-06-28 16:09:20 +0000   
1611  2016-01-08 05:00:14 +0000   

                                                 source  \
1873  <a href="http://twitter.com/download/iphone" r...   
2028  <a href="http://twitter.com/download/iphone" r...   
201   <a href="http://twitter.com/download/iphone" r...   
62    <a href="http://twitter.com/download/iphone" r...   
1611  <a href="http://twitter.com/d

In [14]:
print('rating_denominator mean',twitter_archive['rating_denominator'].mean(),'\n')
print('rating_denominator max',twitter_archive['rating_denominator'].max(),'\n')
print('rating_denominator min',twitter_archive['rating_denominator'].min(),'\n')

rating_denominator mean 10.455432937181664 

rating_denominator max 170 

rating_denominator min 0 



In [15]:
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [16]:
tweet_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1557 entries, ('{"created_at": "Tue Aug 01 00:17:27 +0000 2017"', ' "id": 892177421306343426', ' "id_str": "892177421306343426"', ' "full_text": "This is Tilly. She\'s just checking pup on you. Hopes you\'re doing ok. If not') to ('{"created_at": "Sun Nov 15 22:32:08 +0000 2015"', ' "id": 666020888022790149', ' "id_str": "666020888022790149"', ' "full_text": "Here we have a Japanese Irish Setter. Lost eye in Vietnam (?). Big fan of relaxing on stair. 8/10 would pet https://t.co/BLDqew2Ijj"')
Columns: 122 entries, {"created_at": "Tue Aug 01 16:23:56 +0000 2017" to  "lang": "en"}
dtypes: object(122)
memory usage: 1.5+ MB


In [17]:
api_df_now.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   tweet_id         2354 non-null   int64              
 1   favorites        2354 non-null   int64              
 2   retweets         2354 non-null   int64              
 3   user_followers   2354 non-null   int64              
 4   user_favourites  2354 non-null   int64              
 5   date_time        2354 non-null   datetime64[ns, UTC]
 6   favorite_count   2354 non-null   int64              
dtypes: datetime64[ns, UTC](1), int64(6)
memory usage: 128.9 KB


In [18]:
api_df_now.sample(20)

,tweet_id,favorites,retweets,user_followers,user_favourites,date_time,favorite_count
1325,705975130514706432,3397,842,3200947,114031,2016-03-05 04:36:02+00:00,3397
2116,670420569653809152,668,342,3201009,114031,2015-11-28 01:54:54+00:00,668
587,799422933579902976,8965,2213,3200898,114031,2016-11-18 01:24:14+00:00,8965
969,750101899009982464,3344,959,3200943,114031,2016-07-04 23:00:03+00:00,3344
1116,732005617171337216,16324,6154,3200944,114031,2016-05-16 00:31:53+00:00,16324
2204,668631377374486528,763,349,3201014,114031,2015-11-23 03:25:17+00:00,763
594,798701998996647937,0,9126,3200898,114031,2016-11-16 01:39:30+00:00,0
1701,680913438424612864,2609,705,3200952,114031,2015-12-27 00:49:49+00:00,2609
2198,668655139528511488,562,233,3201014,114031,2015-11-23 04:59:42+00:00,562
354,830956169170665475,8735,1735,3200892,114031,2017-02-13 01:46:03+00:00,8735


# quality

# first: twitter_archive _quality _issues:
   * In twitter_archive data set each of (in_reply_to_status_id,in_reply_to_user_id ,retweeted_status_id,and    retweeted_status_user_id )  must be int64 instead of float64
   * In twitter_archive data set each of( timestamp& retweeted_status_timestamp  )should be datetime instead of object(string)
   * rating_denominator not equal 10 for all values 
   * In several columns null objects are non-null (None to NaN)
   * We only want original ratings (no retweets) that have images.   

# second:image_predictions dataset_quality_issues:
* Missing values from images dataset (2075 rows instead of 2356)
* Some tweet_ids have the same jpg_url
* Some tweets are have 2 different tweet_id one redirect to the other ( it means Dataset contains retweets)
* p1_dog & p1_dog some times are false thats means the algorithm predict the dog as not a breed of dogs

# third:tweet_data dataset:
 * find duplicate values in tweet_id column in excel using vlookup
* I found dublicates 

# Tidiness
* No need to all the informations in images dataset, (tweet_id and jpg_url what matters)
* Dog classifications in four columns: doggo, floofer, pupper, puppo
* Join 'tweet_info' and 'image_predictions' to 'twitter_archive'

# Cleaning¶
 

# in this step we will dublicate data sets and save the clened one in new data set to finally compare between them:

In [19]:
# copy the data frames to clean it:
tweet_data_clean = api_df_now.copy()
twitter_archive_clean = twitter_archive.copy()
image_predictions_clean= image_predictions.copy()

In [20]:
#Add tweet_info and image_predictions to twitter_archive table:
twitter_archive_clean = pd.merge(left=twitter_archive_clean,right=tweet_data_clean, left_on='tweet_id', right_on='tweet_id', how='inner')


In [21]:
twitter_archive_clean = twitter_archive_clean.merge(image_predictions_clean, on='tweet_id', how='inner')

In [22]:
twitter_archive_clean.shape

(2073, 34)

In [23]:
twitter_archive_clean.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,...,1,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,...,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,...,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,...,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,...,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True


In [24]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2073 entries, 0 to 2072
Data columns (total 34 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2073 non-null   int64              
 1   in_reply_to_status_id       23 non-null     float64            
 2   in_reply_to_user_id         23 non-null     float64            
 3   timestamp                   2073 non-null   object             
 4   source                      2073 non-null   object             
 5   text                        2073 non-null   object             
 6   retweeted_status_id         79 non-null     float64            
 7   retweeted_status_user_id    79 non-null     float64            
 8   retweeted_status_timestamp  79 non-null     object             
 9   expanded_urls               2073 non-null   object             
 10  rating_numerator            2073 non-null   int64           

# melting doggo, floofer, pupper and,puppo in one column named as dog_classification

In [25]:
melting_col=['doggo','floofer','pupper','puppo']#melting columns
unchanged_col = [x for x in twitter_archive_clean.columns.tolist() if x not in melting_col]#remaining columns 
print(unchanged_col)
twitter_archive_clean.columns

['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp', 'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator', 'rating_denominator', 'name', 'favorites', 'retweets', 'user_followers', 'user_favourites', 'date_time', 'favorite_count', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog']


Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo',
       'favorites', 'retweets', 'user_followers', 'user_favourites',
       'date_time', 'favorite_count', 'jpg_url', 'img_num', 'p1', 'p1_conf',
       'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'],
      dtype='object')

In [26]:
# Melt the the columns into values
twitter_archive_clean = pd.melt(twitter_archive_clean, id_vars = unchanged_col, value_vars = melting_col, 
                         var_name = 'clasifications', value_name = 'dog_classification')
# Delete column 'stages'
twitter_archive_clean = twitter_archive_clean.drop('clasifications', 1)

# Deleting :null ids /Removing :dublicats id/Deleting tweets with no pictures (tweets only no retweets)

In [27]:
len_1=len(twitter_archive_clean) #length of data before cleaning
twitter_archive_clean = twitter_archive_clean[pd.isnull(twitter_archive_clean.retweeted_status_id)]#remove empty rows

twitter_archive_clean = twitter_archive_clean.drop_duplicates()#remove dublicates

twitter_archive_clean = twitter_archive_clean.dropna(subset = ['jpg_url'])#remove retweets "without image"

len_2=len(twitter_archive_clean)#length of data after cleaning
print("amount of cleaned data",len_1-len_2)

amount of cleaned data 5981


# we no longer need those columns related to retweets

In [28]:
twitter_archive_clean = twitter_archive_clean.drop('retweeted_status_id', 1)
twitter_archive_clean = twitter_archive_clean.drop('retweeted_status_user_id', 1)
twitter_archive_clean = twitter_archive_clean.drop('retweeted_status_timestamp', 1)

In [29]:
twitter_archive_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'favorites', 'retweets', 'user_followers',
       'user_favourites', 'date_time', 'favorite_count', 'jpg_url', 'img_num',
       'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf',
       'p3_dog', 'dog_classification'],
      dtype='object')

In [30]:
twitter_archive_clean.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,...,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,dog_classification
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,...,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,...,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,...,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,...,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,...,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True,None


In [31]:
twitter_archive_clean.dog_classification.value_counts()

None       1994
pupper      212
doggo        74
puppo        23
floofer       8
Name: dog_classification, dtype: int64

In [32]:
#Delete dog_classification duplicates
twitter_archive_clean = twitter_archive_clean.sort_values('dog_classification').drop_duplicates('tweet_id', keep = 'last')

In [33]:
twitter_archive_clean.rating_denominator.value_counts()

10     1976
50        3
80        2
11        2
170       1
150       1
130       1
120       1
110       1
90        1
70        1
40        1
20        1
7         1
2         1
Name: rating_denominator, dtype: int64

In [34]:
twitter_archive_clean.rating_numerator.value_counts()

12      450
10      419
11      396
13      261
9       151
8        95
7        52
14       35
5        33
6        32
3        19
4        16
2         9
1         5
0         2
420       1
24        1
1776      1
27        1
44        1
45        1
50        1
60        1
75        1
80        1
84        1
88        1
99        1
121       1
143       1
144       1
165       1
204       1
26        1
Name: rating_numerator, dtype: int64

# Get rid of image prediction columns# 

In [35]:
#storing fisrt true algorithm with it's level of confidence
p = []#prediction_algorithm
p_conf = [] #confidence level

# Get_prediction_confidence function:
# search the first true algorithm and append it to a list with it's level of confidence
# if flase prediction_algorthm will have a value of none
def get_prediction_confidence(dataframe):
    if dataframe['p1_dog'] == True:
        p.append(dataframe['p1'])
        p_conf.append(dataframe['p1_conf'])
    elif dataframe['p2_dog'] == True:
        p.append(dataframe['p2'])
        p_conf.append(dataframe['p2_conf'])
    elif dataframe['p3_dog'] == True:
        p.append(dataframe['p3'])
        p_conf.append(dataframe['p3_conf'])
    else:
        p.append('none')
        p_conf.append(0)

twitter_archive_clean.apply(get_prediction_confidence, axis=1)
twitter_archive_clean['p'] = p
twitter_archive_clean['p_conf'] = p_conf

In [36]:
twitter_archive_clean.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,...,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,dog_classification,p,p_conf
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,...,False,bagel,0.085851,False,banana,0.076110,False,None,none,0.000000
1465,680055455951884288,NaN,NaN,2015-12-24 16:00:30 +0000,"<a href=""https://about.twitter.com/products/tw...","Meet Sammy. At first I was like ""that's a snow...",https://twitter.com/dog_rates/status/680055455...,10,10,Sammy,...,True,Great_Pyrenees,0.001834,True,Pomeranian,0.000667,True,None,Samoyed,0.995466
1464,680070545539371008,NaN,NaN,2015-12-24 17:00:27 +0000,"<a href=""https://about.twitter.com/products/tw...",Say hello to Emmie. She's trapped in an orname...,https://twitter.com/dog_rates/status/680070545...,9,10,Emmie,...,False,Shih-Tzu,0.121811,True,bubble,0.117820,False,None,Shih-Tzu,0.121811
1463,680085611152338944,NaN,NaN,2015-12-24 18:00:19 +0000,"<a href=""https://about.twitter.com/products/tw...",This is by far the most coordinated series of ...,https://twitter.com/dog_rates/status/680085611...,12,10,by,...,False,apron,0.095023,False,wallet,0.049326,False,None,none,0.000000
1461,680115823365742593,NaN,NaN,2015-12-24 20:00:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Ozzy. He woke up 2 minutes before he h...,https://twitter.com/dog_rates/status/680115823...,9,10,Ozzy,...,True,French_bulldog,0.000544,True,Boston_bull,0.000028,True,None,pug,0.999365


In [37]:
twitter_archive_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'favorites', 'retweets', 'user_followers',
       'user_favourites', 'date_time', 'favorite_count', 'jpg_url', 'img_num',
       'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf',
       'p3_dog', 'dog_classification', 'p', 'p_conf'],
      dtype='object')

In [38]:
# drop the columns of image prediction information
twitter_archive_clean = twitter_archive_clean.drop(['img_num', 'p1', 'p1_conf', 'p1_dog',
                                                    'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'], 1)

In [39]:
twitter_archive_clean.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,favorites,retweets,user_followers,user_favourites,date_time,favorite_count,jpg_url,dog_classification,p,p_conf
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,39467,8853,3200889,114031,2017-08-01 16:23:56+00:00,39467,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,None,none,0.000000
1465,680055455951884288,NaN,NaN,2015-12-24 16:00:30 +0000,"<a href=""https://about.twitter.com/products/tw...","Meet Sammy. At first I was like ""that's a snow...",https://twitter.com/dog_rates/status/680055455...,10,10,Sammy,18278,8067,3200953,114031,2015-12-24 16:00:30+00:00,18278,https://pbs.twimg.com/media/CW-ZRC_WQAAyFrL.jpg,None,Samoyed,0.995466
1464,680070545539371008,NaN,NaN,2015-12-24 17:00:27 +0000,"<a href=""https://about.twitter.com/products/tw...",Say hello to Emmie. She's trapped in an orname...,https://twitter.com/dog_rates/status/680070545...,9,10,Emmie,2818,994,3200953,114031,2015-12-24 17:00:27+00:00,2818,https://pbs.twimg.com/media/CW-dU34WQAANBGy.jpg,None,Shih-Tzu,0.121811
1463,680085611152338944,NaN,NaN,2015-12-24 18:00:19 +0000,"<a href=""https://about.twitter.com/products/tw...",This is by far the most coordinated series of ...,https://twitter.com/dog_rates/status/680085611...,12,10,by,13959,10034,3200953,114031,2015-12-24 18:00:19+00:00,13959,https://pbs.twimg.com/media/CXAiiHUWkAIN_28.jpg,None,none,0.000000
1461,680115823365742593,NaN,NaN,2015-12-24 20:00:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Ozzy. He woke up 2 minutes before he h...,https://twitter.com/dog_rates/status/680115823...,9,10,Ozzy,2972,1028,3200953,114031,2015-12-24 20:00:22+00:00,2972,https://pbs.twimg.com/media/CXBBurSWMAELewi.jpg,None,pug,0.999365


# drope some columns

now The following columns 
(‘in_reply_to_status_id’, ‘in_reply_to_user_id’, 
 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp')
will be utilized to shed the retweet and replies from our datasets and then will be dropped.
After dropping the replies and retweets, check the image_predictions table for extra tweet ids not in the archive table.


In [40]:
twitter_archive_clean = twitter_archive_clean.drop(['in_reply_to_status_id', 'in_reply_to_user_id', 'user_favourites'], 1)

In [41]:
twitter_archive_clean['source'] = twitter_archive_clean['source'].apply(lambda x: re.findall(r'>(.*)<', x)[0])

In [42]:
twitter_archive_clean.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,favorites,retweets,user_followers,date_time,favorite_count,jpg_url,dog_classification,p,p_conf
0,892420643555336193,2017-08-01 16:23:56 +0000,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,39467,8853,3200889,2017-08-01 16:23:56+00:00,39467,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,None,none,0.000000
1465,680055455951884288,2015-12-24 16:00:30 +0000,TweetDeck,"Meet Sammy. At first I was like ""that's a snow...",https://twitter.com/dog_rates/status/680055455...,10,10,Sammy,18278,8067,3200953,2015-12-24 16:00:30+00:00,18278,https://pbs.twimg.com/media/CW-ZRC_WQAAyFrL.jpg,None,Samoyed,0.995466
1464,680070545539371008,2015-12-24 17:00:27 +0000,TweetDeck,Say hello to Emmie. She's trapped in an orname...,https://twitter.com/dog_rates/status/680070545...,9,10,Emmie,2818,994,3200953,2015-12-24 17:00:27+00:00,2818,https://pbs.twimg.com/media/CW-dU34WQAANBGy.jpg,None,Shih-Tzu,0.121811
1463,680085611152338944,2015-12-24 18:00:19 +0000,TweetDeck,This is by far the most coordinated series of ...,https://twitter.com/dog_rates/status/680085611...,12,10,by,13959,10034,3200953,2015-12-24 18:00:19+00:00,13959,https://pbs.twimg.com/media/CXAiiHUWkAIN_28.jpg,None,none,0.000000
1461,680115823365742593,2015-12-24 20:00:22 +0000,Twitter for iPhone,This is Ozzy. He woke up 2 minutes before he h...,https://twitter.com/dog_rates/status/680115823...,9,10,Ozzy,2972,1028,3200953,2015-12-24 20:00:22+00:00,2972,https://pbs.twimg.com/media/CXBBurSWMAELewi.jpg,None,pug,0.999365


In [43]:
twitter_archive_clean.columns

Index(['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
       'rating_numerator', 'rating_denominator', 'name', 'favorites',
       'retweets', 'user_followers', 'date_time', 'favorite_count', 'jpg_url',
       'dog_classification', 'p', 'p_conf'],
      dtype='object')

# Fix rating numerator and denominators that are not actually ratings

For the denominator; any value below or above 10 is suspected

In [44]:
twitter_archive_clean['rating_denominator']=10

In [45]:
twitter_archive_clean.rating_denominator.value_counts()

10    1994
Name: rating_denominator, dtype: int64

# 

In [46]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 7108
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            1994 non-null   int64              
 1   timestamp           1994 non-null   object             
 2   source              1994 non-null   object             
 3   text                1994 non-null   object             
 4   expanded_urls       1994 non-null   object             
 5   rating_numerator    1994 non-null   int64              
 6   rating_denominator  1994 non-null   int64              
 7   name                1994 non-null   object             
 8   favorites           1994 non-null   int64              
 9   retweets            1994 non-null   int64              
 10  user_followers      1994 non-null   int64              
 11  date_time           1994 non-null   datetime64[ns, UTC]
 12  favorite_count      1994 non-null 

In [50]:
twitter_archive_clean.drop(twitter_archive_clean.columns[twitter_archive_clean.columns.str.contains('Unnamed',case = False)],axis = 1)
twitter_archive_clean.to_csv('twitter_archive_master.csv', encoding = 'utf-8', index=False)

In [51]:
twitter_archive_clean = pd.read_csv('twitter_archive_master.csv')
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1994 entries, 0 to 1993
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tweet_id            1994 non-null   int64  
 1   timestamp           1994 non-null   object 
 2   source              1994 non-null   object 
 3   text                1994 non-null   object 
 4   expanded_urls       1994 non-null   object 
 5   rating_numerator    1994 non-null   int64  
 6   rating_denominator  1994 non-null   int64  
 7   name                1994 non-null   object 
 8   favorites           1994 non-null   int64  
 9   retweets            1994 non-null   int64  
 10  user_followers      1994 non-null   int64  
 11  date_time           1994 non-null   object 
 12  favorite_count      1994 non-null   int64  
 13  jpg_url             1994 non-null   object 
 14  dog_classification  1994 non-null   object 
 15  p                   1994 non-null   object 
 16  p_conf